IMPORTING LIBRARIES 📚

In [214]:
from sqlalchemy import create_engine
from sqlalchemy import URL
from sqlalchemy import text
from urllib.parse import quote_plus
from sqlalchemy.types import Integer, VARCHAR, CHAR
from sqlalchemy import ForeignKeyConstraint
import pandas as pd 

Connection and CREATE SCHEMA 📝

In [217]:
url_object = URL.create(
    "mysql+mysqlconnector",
    username="------",
    password="-----",
    host="localhost",
)

In [218]:
engine = create_engine(url_object)

In [219]:
with engine.connect() as conn : 
    conn.execute(text('CREATE Schema football;'))
    # conn.execute(text('DROP schema imdb250;'))

Read Clubs Data 📗

In [220]:
df_club = pd.read_csv('Phase1\data\club.csv', index_col=0)
df_club_players = pd.read_csv('Phase1\data\club_players.csv', index_col=0)
df_club_table_league = pd.read_csv('Phase1\data\club_table_league.csv', index_col=0)

In [221]:
df_club_table_league.head(3)

,League,Season,Club,Rank
0,Premier League,2015,Leicester,1.0
1,Premier League,2015,Arsenal,2.0
2,Premier League,2015,Tottenham,3.0


Read Players Data 📗

In [222]:
df_players_datas = pd.read_csv('Phase1\data\players_datas.csv', index_col=0)
df_players_stats = pd.read_csv('Phase1\data\players_stats.csv', index_col=0)
df_players_transfers = pd.read_csv('Phase1\data\players_transfers.csv', index_col=0)

In [223]:
df_players_datas.head(3)

,shirt_number,given_name,full_name,date_of_birth,citizenship,place_of_birth,caps,goals,other_positions,outfitter,contract_expires,agent,foot,contract_Joined,height,current_club,date_of_last_contract,players_link
player_id,,,,,,,,,,,,,,,,,,
465955,#90,Marcin Bulka,Marcin Bułka,"Oct 4, 1999",Poland,Płock,1.0,0.0,NaN,NaN,"Jun 30, 2026",AIS Football,Current club:,"Jul 1, 2022",199.0,OGC Nice,NaN,https://www.transfermarkt.com/marcin-bulka/pro...
2998,NaN,Robert Huth,NaN,"Aug 18, 1984",Germany,Berlin,19.0,2.0,Right-Back,NaN,-,Christian Nilles,NaN,"Jul 1, 2018",NaN,Retired,NaN,https://www.transfermarkt.com/robert-huth/prof...
406633,NaN,Nya Kirby,Nya Jerome Kirby,"Jan 31, 2000",England,London,10.0,1.0,"Attacking Midfield, Left Midfield",NaN,-,Relatives,right,"Jul 1, 2022",174.0,Without Club,NaN,https://www.transfermarkt.com/nya-kirby/profil...


CREATE TABLE Country ⚽️

In [224]:
table_country = df_club.loc[:, ['CountryID', 'Country']].drop_duplicates()
table_country = table_country.rename(columns={'CountryID': 'country_id', 'Country': 'country_name'}).set_index('country_id')
table_country

,country_name
country_id,
189,England
40,Germany
75,Italy
50,France
157,Spain


In [225]:
with engine.connect() as conn:
    table_country.to_sql(name='country', con=conn, schema='football', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'country_id', dtype={'country_id': Integer(), 'country_name': VARCHAR(15)})


CREATE TABLE League ⚽️

In [226]:
table_league = df_club.loc[:, ['League', 'CountryID']].drop_duplicates().assign(league_id=range(1, 6)).set_index('league_id')
table_league = table_league.rename(columns={'League': 'league_name', 'CountryID': 'country_id'})
table_league

,league_name,country_id
league_id,,
1,Premier League,189
2,Bundesliga,40
3,Serie A,75
4,Ligue 1,50
5,LaLiga,157


In [227]:
query_league = text("""
CREATE TABLE league (
league_id INT NOT NULL AUTO_INCREMENT,
league_name VARCHAR(30) NOT NULL,
country_id INT NOT NULL,
PRIMARY KEY (league_id),
FOREIGN KEY (country_id) REFERENCES country(country_id)
);""")

In [228]:
with engine.connect() as conn:
    conn.execute(text('USE football;'))
    conn.execute(query_league)

In [229]:
with engine.connect() as conn:
    table_league.to_sql(name='league', con=conn, schema='football', if_exists='append', chunksize=50, method='multi')

CREATE TABLE Club ⚽️

In [230]:
table_club = df_club.loc[:, ['ClubID', 'Club', 'Stadium', 'StadiumCap']].drop_duplicates()
table_club = table_club.rename(columns={'ClubID': 'club_id', 'Club': 'club_name', 'Stadium': 'stadium_name', 'StadiumCap': 'stadium_capacity'}).set_index('club_id')
table_club.head(3)

,club_name,stadium_name,stadium_capacity
club_id,,,
631,Chelsea FC,Stamford Bridge,40.853 Seats
281,Manchester City,Etihad Stadium,55.017 Seats
11,Arsenal FC,Emirates Stadium,60.704 Seats


In [231]:
with engine.connect() as conn:
    table_club.to_sql(name='club', con=conn, schema='football', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'club_id', dtype={'club_id': Integer(), 'club_name': VARCHAR(30), 
                                                         'stadium_name': VARCHAR(30), 'stadium_capacity': VARCHAR(20)})


CREATE TABLE Season ⚽️

In [232]:
table_season = df_club.loc[:, ['Season']].drop_duplicates()
table_season = table_season.assign(season_id=range(1, len(table_season) + 1)).set_index('season_id').rename(columns={'Season': 'year'})
table_season

,year
season_id,
1,2015
2,2016
3,2017
4,2018
5,2019
6,2020
7,2021


In [233]:
with engine.connect() as conn:
    table_season.to_sql(name='season', con=conn, schema='football', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'season_id', dtype={'season_id': Integer(), 'year': Integer()})


CREATE TABLE Season_info ⚽️

In [234]:
query_season_info = text("""
CREATE TABLE season_info (
season_info_id INT NOT NULL AUTO_INCREMENT,
season_id INT NOT NULL,
league_id INT NOT NULL,
club_id INT NOT NULL,
club_rank INT,
squad INT,
avg_age FLOAT,
foreign_players INT,
avg_marketValue VARCHAR(12),
total_marketValue VARCHAR(12),
income VARCHAR(12),
overbalance VARCHAR(12),
expenditure VARCHAR(12),
PRIMARY KEY (season_info_id),
FOREIGN KEY (season_id) REFERENCES season(season_id),
FOREIGN KEY (league_id) REFERENCES league(league_id),
FOREIGN KEY (club_id) REFERENCES club(club_id)
);""")

In [235]:
# with engine.connect() as conn:
#     conn.execute(text('USE football;'))
#     conn.execute(query_season_info)

In [ ]:
# league_dict = dict(zip(table_league['league_name'], table_league.index))
# league_dict

In [ ]:
# table_season_info.head(1)

In [ ]:
# table_season_info = df_club.loc[:, ['Season', 'League', 'Club', 'ClubID', 'Squad', 
#                                     'avgAge', 'Foreigners', 'avgMarketValue', 'totalMarketValue', 
#                                     'Club_income', 'Club_expenditure', 'Club_OverallBalance']]
                                    
# table_season_info = table_season_info.merge(df_club_table_league, on=['League', 'Season', 'Club'], how='left')
# table_season_info['League'] = table_season_info['League'].map(league_dict)
# table_season_info

In [236]:
# with engine.connect() as conn:
#     table_season.to_sql(name='season', con=conn, schema='football', if_exists='append', chunksize=50, method='multi')

CREATE TABLE Coach ⚽️

In [ ]:
# mohammad

CREATE TABLE Coach_Season_info ⚽️

In [ ]:
# mohammad

CREATE TABLE Victories ⚽️

In [ ]:
# mohammad

CREATE TABLE Victories_Season_info ⚽️

In [ ]:
# mohammad

CREATE TABLE Positions ⚽️

In [ ]:
# Ali

CREATE TABLE Players ⚽️

In [ ]:
# Milad

CREATE TABLE Players_Season_info ⚽️

In [ ]:
# ALi

CREATE TABLE Outfitters ⚽️

In [ ]:
# milad

CREATE TABLE Player_info ⚽️

In [ ]:
# zahra

CREATE TABLE Outfitter_Player_info ⚽️

In [ ]:
# Zhara

CREATE TABLE Stats ⚽️

In [ ]:
# Zahra

CREATE TABLE Transfer ⚽️

In [ ]:
# milad